In [2]:
import json
import numpy as np
import pandas as pd

In [171]:
def pointsAboveReplacement(file):
    with open(file, "r") as infile:
        data = json.load(infile)
    for i, player in enumerate(data):
        if i < (len(data)-1):
            data[i].update({"points_above_replacement": player["togga_score"] - data[i+1]["togga_score"]})   
    with open(file, 'w') as json_data:
        json.dump(data, json_data)

for position in ['goalies', 'defense', 'midfield', 'forwards']:
    for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
        file = 'Data/Sorted_Positions/sorted_%s_%s+%s.json' % (position, year[:4], year[-2:])
        pointsAboveReplacement(file)



In [172]:
with open(file, "r") as infile:
        data = json.load(infile)

points_above = np.array([(x["name"], x["points_above_replacement"]) for x in data[:-1]])
#print(points_above)


In [173]:
# Removed ontarget_scoring_att
relevant_forward_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                         'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 
                         'goals_scored', 'key_passes', 'accurateCrosses_total', 'aerialWon_total', 'assists', 
                         'points_above_replacement', 'name', 'age', 'ontarget_scoring_att']

relevant_midfield_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                          'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 'ontarget_scoring_att',
                          'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 'aerialWon_total', 
                          'assists', 'points_above_replacement', 'name', 'age']

relevant_defense_keys = ['goals_conceded', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total',
                         'red_cards', 'minutes', 'successful_dribbles', 'own_goals', 'name', 'ontarget_scoring_att',
                         'interception_total', 'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 
                         'aerialWon_total', 'assists', 'points_above_replacement', 'clearance_total', 'age']

relevant_goalie_keys = ['goals_conceded', 'saves', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'red_cards', 
                        'minutes', 'own_goals', 'interception_total', 'key_passes', 'clean_sheets', 'aerialWon_total', 
                        'penalties_saved', 'points_above_replacement', 'name', 'clearance_total', 'age']

def removeExtraStats(file, relevant_keys):
    with open(file, "r") as infile:
        data = json.load(infile)
    new_data = []
    for i in range(len(data)):
        values = [{key: data[i].get(key)} for key in relevant_keys]
        item = dict(pair for d in values for pair in d.items())
        new_data.append(item)
    with open(file, 'w') as json_data:
        json.dump(new_data, json_data)
        
for position in ['goalies', 'defense', 'midfield', 'forwards']:
    for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
        file = 'Data/Sorted_Positions/sorted_%s_%s+%s.json' % (position, year[:4], year[-2:])
        if position == 'goalies':
            keys = relevant_goalie_keys
        elif position == 'defense':
            keys = relevant_defense_keys
        elif position == 'midfield':
            keys = relevant_midfield_keys
        else:
            keys = relevant_forward_keys
        removeExtraStats(file, keys)

In [177]:
with open('Data/Sorted_Positions/sorted_forwards_2016+17.json', "r") as infile:
    data = json.load(infile)
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 38 entries

In [179]:
#df

In [175]:
with open('Data/Sorted_Positions/sorted_midfield_2016+17.json', "r") as infile:
    data = json.load(infile)
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 125 entries

In [201]:
#df

In [203]:
for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
    file = 'Data/Sorted_Positions/sorted_defense_%s+%s.json' % (year[:4], year[-2:])
    with open(file, "r") as infile:
        data = json.load(infile)
    for player in data:
        if not player.get("ontarget_scoring_att"):
            player["ontarget_scoring_att"] = 0
    with open(file, 'w') as json_data:
        json.dump(data, json_data)

In [205]:
with open('Data/Sorted_Positions/sorted_defense_2016+17.json', "r") as infile:
    data = json.load(infile)
    
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 86 entries

In [206]:
df

,accurateCrosses_total,aerialWon_total,age,apps,assists,clean_sheets,clearance_total,dispossessed_total,goals_conceded,goals_scored,...,minutes,name,ontarget_scoring_att,own_goals,points_above_replacement,red_cards,successful_dribbles,tackle_total,togga_score,yellow_cards
0,17,73,26,31,5,15,77,30,19,6,...,2693,Marcos Alonso,16,0,52.00,0,21,58,445.25,2
1,7,81,27,38,5,16,119,9,33,1,...,3420,César Azpilicueta,0,0,21.75,0,12,85,393.25,4
2,13,60,27,33,6,14,94,28,21,0,...,2701,Kyle Walker,0,0,30.25,0,35,72,371.50,8
3,0,91,31,37,0,17,167,7,29,6,...,3296,Gary Cahill,10,1,3.00,0,4,52,341.25,5
4,26,69,31,36,4,8,143,11,57,2,...,3103,Christian Fuchs,0,0,5.00,0,13,97,338.25,10
5,1,117,29,30,2,9,189,6,28,1,...,2592,Nicolás Otamendi,0,0,3.75,0,11,69,333.25,9
6,41,22,31,28,3,12,58,14,20,1,...,2483,Antonio Valencia,0,0,1.25,0,37,67,329.50,5
7,33,27,32,32,5,11,65,12,34,2,...,2819,Leighton Baines,0,0,7.75,0,12,63,328.25,4
8,1,152,24,35,1,10,260,7,49,2,...,3150,Michael Keane,8,0,2.50,0,5,28,320.50,4
9,1,106,30,33,0,15,230,9,21,0,...,2894,Jan Vertonghen,0,0,0.50,0,26,43,318.00,5
